In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

#ipython很好用，但是如果在ipython里已经import过的模块修改后需要重新reload就需要这样
#在执行用户代码前，重新装入软件的扩展和模块。
%load_ext autoreload   
#autoreload 2：装入所有 %aimport 不包含的模块。
%autoreload 2          

np.random.seed(1)      #指定随机种子

In [2]:
def zero_pad(X,pad):
    """
    把数据集X的图像边界全部使用0来扩充pad个宽度和高度。
    
    参数：
        X - 图像数据集，维度为（样本数，图像高度，图像宽度，图像通道数）
        pad - 整数，每个图像在垂直和水平维度上的填充量
    返回：
        X_paded - 扩充后的图像数据集，维度为（样本数，图像高度 + 2*pad，图像宽度 + 2*pad，图像通道数）
    
    """
    X_paded = np.pad(X,(
                        (0,0),     #样本数不填充
                        (pad,pad), #图像高度，上/下面都填充pad个(x,y)
                        (pad,pad), #图像宽度，上/下面都填充pad个(x,y)
                        (0,0))     #图像轨道数，不填充
                        ,'constant', constant_values=0)    #填充方式为constant,填充值为0
    
    return X_paded

#test
# np.random.seed(1)
# x = np.random.randn(4,3,3,2)
# x_paded = zero_pad(x,2)
# #查看信息
# print ("x.shape =", x.shape)
# print ("x_paded.shape =", x_paded.shape)
# print ("x[1, 1] =", x[1, 1])
# print ("x_paded[1, 1] =", x_paded[1, 1])

# #绘制图
# fig , axarr = plt.subplots(1,2)  #一行两列
# axarr[0].set_title('x')
# axarr[0].imshow(x[0,:,:,0])
# axarr[1].set_title('x_paded')
# axarr[1].imshow(x_paded[0,:,:,0])

In [3]:
def conv_single_step(a_slice_prev,W,b):
    """
    在前一层的激活输出的一个片段上应用一个由参数W定义的过滤器。
    这里切片大小和过滤器大小相同
    
    参数：
        a_slice_prev - 输入数据的一个片段，维度为（过滤器大小，过滤器大小，上一通道数）
        W - 权重参数，包含在了一个矩阵中，维度为（过滤器大小，过滤器大小，上一通道数）
        b - 偏置参数，包含在了一个矩阵中，维度为（1,1,1）
        
    返回：
        Z - 在输入数据的片X上卷积滑动窗口（w，b）的结果。
    """
    s = np.multiply(a_slice_prev,W) + b
    Z = np.sum(s)
    
    return Z
#test
# np.random.seed(1)

#这里切片大小和过滤器大小相同
# a_slice_prev = np.random.randn(4,4,3)
# W = np.random.randn(4,4,3)
# b = np.random.randn(1,1,1)

# Z = conv_single_step(a_slice_prev,W,b)

# print("Z = " + str(Z))

In [4]:
def conv_forward(A_prev, W, b, hparameters):
    """
    实现卷积函数的前向传播
    
    参数：
        A_prev - 上一层的激活输出矩阵，维度为(m, n_H_prev, n_W_prev, n_C_prev)，（样本数量，上一层图像的高度，上一层图像的宽度，上一层过滤器数量）
        W - 权重矩阵，维度为(f, f, n_C_prev, n_C)，（过滤器大小，过滤器大小，上一层的过滤器数量，这一层的过滤器数量）
        b - 偏置矩阵，维度为(1, 1, 1, n_C)，（1,1,1,这一层的过滤器数量）
        hparameters - 包含了"stride"与 "pad"的超参数字典。
    
    返回：
        Z - 卷积输出，维度为(m, n_H, n_W, n_C)，（样本数，图像的高度，图像的宽度，过滤器数量）
        cache - 缓存了一些反向传播函数conv_backward()需要的一些数据
    """
    
    #获取来自上一层的基本信息
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    #获取权重的基本信息
    (f, f, n_C_prev, n_C) = W.shape
    
    #获取超参数parameters的值
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    #计算卷积后的图像的宽度高度，使用int来进行板除
    n_H = int((n_H_prev + 2* pad - f) / stride) + 1
    n_W = int((n_W_prev + 2* pad - f) / stride) + 1
    
    #使用0来初始化卷积的输出
    Z = np.zeros((m,n_H,n_W,n_C))
    
    #通过A_prev创建填充了的A_prev_pad
    A_prev_pad = zero_pad(A_prev,pad)
    
    for i in range(m):                         #遍历样本
        a_prev_pad = A_prev_pad[i]             #选择填充过后的第i个激活矩阵，第i个样本
        for h in range(n_H):                   #图片的高度，不是n_H_prev，不用考虑参数的问题了（已经知道遍历的下界是多少了）
             #3rd->纵向移动   
            for w in range(n_W):               #图片的宽度
                #2nd->横向移动
                for c in range(n_C):           #filter的个数
                    #1st->先算filter
                    #计算切片的范围
                    vert_start = h*stride      #竖向开始的位置
                    vert_end   = vert_start + f#竖向结束的位置
                    horiz_start= w*stride      #纵向开始的位置
                    horiz_end  = horiz_start + f#纵向结束的位置
                    
                    #取出切片，第三个维度全部都要取，对应n_C_prev
                    a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]  
                    
                    #执行单步卷积，W的第四个维度对应filter的编号，对应Z的第四个维度
                    Z[i,h,w,c] = conv_single_step(a_slice_prev,W[:,:,:,c],b[0,0,0,c])
                    
    #数据处理完毕，检验
    assert(Z.shape == (m,n_H,n_W,n_C))
    
    #储存一些缓存值
    cache = (A_prev,W,b,hparameters)
    
    return (Z, cache)

#test
# np.random.seed(1)

# A_prev = np.random.randn(10,4,4,3)
# W = np.random.randn(2,2,3,8)
# b = np.random.randn(1,1,1,8)

# hparameters = {"pad" : 2, "stride": 1}

# Z , cache_conv = conv_forward(A_prev,W,b,hparameters)

# print("np.mean(Z) = ", np.mean(Z))
# print("cache_conv[0][1][2][3] =", cache_conv[0][1][2][3])

In [20]:
def pool_forward(A_prev,hparameters,mode="max"):
    """
    实现池化层的前向传播
    
    参数：
        A_prev - 输入数据，维度为(m, n_H_prev, n_W_prev, n_C_prev)
        hparameters - 包含了 "f" 和 "stride"的超参数字典
        mode - 模式选择【"max" | "average"】
        
    返回：
        A - 池化层的输出，维度为 (m, n_H, n_W, n_C)
        cache - 存储了一些反向传播需要用到的值，包含了输入和超参数的字典。
    """
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    stride = hparameters["stride"]
    f = hparameters["f"]
    
    n_H = int((n_H_prev - f)/ stride) + 1
    n_W = int((n_W_prev - f)/ stride) + 1
    n_C = n_C_prev
    
    A = np.zeros((m,n_H,n_W,n_C))
    
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                vert_start = h * stride
                vert_end   = vert_start + f
                horiz_start= w * stride
                horiz_end  = horiz_start + f
                for c in range(n_C):
                    if mode == "max":
                        A[i, h, w, c] = np.max(A_prev[i,vert_start:vert_end,horiz_start:horiz_end,c])
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(A_prev[i,vert_start:vert_end,horiz_start:horiz_end,c])
    
    assert(A.shape == (m, n_H, n_W, n_C))
    
    cache = (A_prev, hparameters)
    
    return A,cache
                        
#test
# np.random.seed(1)
# A_prev = np.random.randn(2,4,4,3)
# hparameters = {"f":4 , "stride":1}

# A , cache = pool_forward(A_prev,hparameters,mode="max")
# A, cache = pool_forward(A_prev, hparameters)
# print("mode = max")
# print("A =", A)
# print("----------------------------")
# A, cache = pool_forward(A_prev, hparameters, mode = "average")
# print("mode = average")
# print("A =", A)

In [21]:
def conv_backward(dZ,cache):
    """
    实现卷积层的反向传播
    
    参数：
        dZ - 卷积层的输出Z的 梯度，维度为(m, n_H, n_W, n_C)
        cache - 反向传播所需要的参数，conv_forward()的输出之一
        
    返回：
        dA_prev - 卷积层的输入（A_prev）的梯度值，维度为(m, n_H_prev, n_W_prev, n_C_prev)
        dW - 卷积层的权值的梯度，维度为(f,f,n_C_prev,n_C)
        db - 卷积层的偏置的梯度，维度为（1,1,1,n_C）
    
    """
    #获取cache的值
    (A_prev, W, b, hparameters) = cache
    
    #获取信息
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    #获取dz的信息
    (m, n_H, n_W, n_C) = dZ.shape
    
    #获取dw的信息
    (f, f, n_C_prev, n_C) = W.shape
    
    #获取超参数
    pad = hparameters["pad"]
    stride = hparameters["stride"]
    
    #初始化各个梯度的结构
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1,1,1,n_C))
    
    #填充
    A_prev_pad = zero_pad(A_prev,pad)
    dA_prev_pad = zero_pad(dA_prev,pad)
    
    for i in range(m):
        #选择第i个样本, 降了一个维度
        a_prev_pad = A_prev_pad[i]
        da_prev_pad = dA_prev_pad[i]
    
        #现在处理数据
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end   = vert_start + f
                    horiz_start= w * stride
                    horiz_end  = horiz_start+ f
                
                    a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end,:]
                    
                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end,:] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
                  
        #把非填充部分的数据拿出来        
        dA_prev[i,:,:,:] = da_prev_pad[pad:-pad,pad:-pad,:]
        
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))
    
    return (dA_prev, dW, db)

#test
# np.random.seed(1)
# #初始化参数
# A_prev = np.random.randn(10,4,4,3)
# W = np.random.randn(2,2,3,8)
# b = np.random.randn(1,1,1,8)
# hparameters = {"pad" : 2, "stride": 1}

# #前向传播
# Z , cache_conv = conv_forward(A_prev,W,b,hparameters)
# #反向传播
# dA , dW , db = conv_backward(Z,cache_conv)
# print("dA_mean =", np.mean(dA))
# print("dW_mean =", np.mean(dW))
# print("db_mean =", np.mean(db))

In [22]:
def create_mask_from_window(x):
    """
    从输入矩阵中创建掩码，以保存最大值的矩阵的位置。
    
    参数：
        x - 一个维度为(f,f)的矩阵
        
    返回：
        mask - 包含x的最大值的位置的矩阵
    """
    mask = x == np.max(x) #第一个=是赋值;第二个是等号，是判断
    
    return mask

#test
# np.random.seed(1)

# x = np.random.randn(2,3)

# mask = create_mask_from_window(x)

# print("x = " + str(x)) 
# print("mask = " + str(mask))\

def distribute_value(dz,shape):
    """
    给定一个值，为按矩阵大小平均分配到每一个矩阵位置中。
    
    参数：
        dz - 输入的实数
        shape - 元组，两个值，分别为n_H , n_W
        
    返回：
        a - 已经分配好了值的矩阵，里面的值全部一样。
    
    """
    #获取矩阵大小
    
    #获取数值
    average = dz / (shape[0] * shape[1])
    
    #填充入矩阵
    a= np.ones(shape) * average
    
    return a

#test
# dz = 2
# shape = (2,2)

# a = distribute_value(dz,shape)
# print("a = " + str(a))

In [23]:
def pool_backward(dA,cache,mode = "max"):
    """
    实现池化层的反向传播
    
    参数:
        dA - 池化层的输出的梯度，和池化层的输出的维度一样
        cache - 池化层前向传播时所存储的参数。
        mode - 模式选择，【"max" | "average"】
        
    返回：
        dA_prev - 池化层的输入的梯度，和A_prev的维度相同
    
    """
    
    #获取cache的值
    (A_prev, hparameters) = cache
    
    #获取parameters的值
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    #获取A_prev和dA的基本信息
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (m, n_H, n_W, n_C) = dA.shape
    
    #初始化输出结构
    dA_prev = np.zeros_like(A_prev)
    
    #开始处理数据
    for i in range(m):
        a_prev = A_prev[i]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end   = vert_start + f
                    horiz_start= w * stride
                    horiz_end  = horiz_start+ f
                    
                    if mode == "max":
                        #切片
                        a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end,c]
                        #创建代码
                        mask = create_mask_from_window(a_prev_slice)
                        #计算dA_prev
                        dA_prev[i,vert_start:vert_end, horiz_start:horiz_end,c] += np.multiply(mask, dA[i,h,w,c])
                        
                    elif mode == "average":
                        #获取dA的值
                        da = dA[i,h,w,c]
                        #定义过滤器的大小
                        shape = (f,f)
                        #平均分配
                        dA_prev[i,vert_start:vert_end, horiz_start:horiz_end,c] += distribute_value(da,shape)
    
    #检验
    assert(dA_prev.shape == (m,n_H_prev,n_W_prev,n_C_prev))
    
    return dA_prev

#test
# np.random.seed(1)
# A_prev = np.random.randn(5, 5, 3, 2)
# hparameters = {"stride" : 1, "f": 2}
# A, cache = pool_forward(A_prev, hparameters)
# dA = np.random.randn(5, 4, 2, 2)

# dA_prev = pool_backward(dA, cache, mode = "max")
# print("mode = max")
# print('mean of dA = ', np.mean(dA))
# print('dA_prev[1,1] = ', dA_prev[1,1])  
# print()
# dA_prev = pool_backward(dA, cache, mode = "average")
# print("mode = average")
# print('mean of dA = ', np.mean(dA))
# print('dA_prev[1,1] = ', dA_prev[1,1]) 

In [24]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow.compat.v1 as tf 
from tensorflow.python.framework import ops
tf.disable_v2_behavior()

import cnn_utils

%matplotlib inline
np.random.seed(1)

In [29]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    """
    为session创建占位符
    
    参数：
        n_H0 - 实数，输入图像的高度
        n_W0 - 实数，输入图像的宽度
        n_C0 - 实数，输入的通道数
        n_y  - 实数，分类数
        
    输出：
        X - 输入数据的占位符，维度为[None, n_H0, n_W0, n_C0]，类型为"float"
        Y - 输入数据的标签的占位符，维度为[None, n_y]，维度为"float"
    """
    X = tf.placeholder(tf.float32,[None,n_H0,n_W0,n_C0])
    Y = tf.placeholder(tf.float32,[None,n_y])
    
    return X,Y

#test
# X , Y = create_placeholders(64,64,3,6)
# print ("X = " + str(X))
# print ("Y = " + str(Y))

X = Tensor("Placeholder:0", shape=(?, 64, 64, 3), dtype=float32)
Y = Tensor("Placeholder_1:0", shape=(?, 6), dtype=float32)


In [32]:
def initialize_parameters():
    """
    初始化权值矩阵，这里我们把权值矩阵硬编码：
    W1 : [4, 4, 3, 8]
    W2 : [2, 2, 8, 16]
    
    返回：
        包含了tensor类型的W1、W2的字典
    """
    tf.set_random_seed(1)
    
    W1 = tf.get_variable("W1",[4,4,3,8],initializer=tf.glorot_uniform_initializer(seed=0))
    W2 = tf.get_variable("W2",[2,2,8,16],initializer=tf.glorot_uniform_initializer(seed=0))
    
    parameters = {"W1":W1,
                  "W2":W2}
    
    return parameters

#test
# tf.reset_default_graph()
# with tf.Session() as sess_test:
#     parameters = initialize_parameters()
#     init = tf.global_variables_initializer()
#     sess_test.run(init)
#     print("W1 = " + str(parameters["W1"].eval()[1,1,1]))
#     print("W2 = " + str(parameters["W2"].eval()[1,1,1]))
    
#     sess_test.close()

W1 = [ 0.00131723  0.1417614  -0.04434952  0.09197326  0.14984085 -0.03514394
 -0.06847463  0.05245192]
W2 = [-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]


In [44]:
def forward_propagation(X,parameters):
    """
    实现前向传播
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    参数：
        X - 输入数据的placeholder，维度为(输入节点数量，样本数量)
        parameters - 包含了“W1”和“W2”的python字典。
        
    返回：
        Z3 - 最后一个LINEAR节点的输出
    
    """
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
    #Conv2d : 步伐: 1, 填充方式为: "same"
    Z1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding="SAME")
    
    #RELU:
    A1 = tf.nn.relu(Z1)
    
    #Max pool : 窗口大小：8x8，步伐：8x8，填充方式：“SAME”
    P1 = tf.nn.max_pool(A1,ksize=[1,8,8,1],strides=[1,8,8,1],padding="SAME")
    
    #Conv2d : 步伐：1，填充方式：“SAME”
    Z2 = tf.nn.conv2d(P1,W2,strides=[1,1,1,1],padding="SAME")
    
    #RELU:
    A2 = tf.nn.relu(Z2)
    
    #Max pool : 窗口大小：8x8，步伐：8x8，填充方式：“SAME”
    P2 = tf.nn.max_pool(A2,ksize=[1,8,8,1],strides=[1,8,8,1],padding="SAME")
    
    #一维化上一层的输出
    #P = P2[:,-1]    
    
    #全连接层（FC）：使用没有非线性激活函数的全连接层
    Z3 = tf.keras.layers.Dense(6, activation = None)(P)
    
    return Z3

#test
# tf.reset_default_graph()
# np.random.seed(1)

# with tf.Session() as sess_test:
#     X,Y = create_placeholders(64,64,3,6)
#     parameters = initialize_parameters()
#     Z3 = forward_propagation(X,parameters)
    
#     init = tf.global_variables_initializer()
#     sess_test.run(init)
    
#     a = sess_test.run(Z3,{X: np.random.randn(2,64,64,3), Y: np.random.randn(2,6)})
#     print("Z3 = " + str(a))
    
#     sess_test.close()

SyntaxError: invalid syntax (<ipython-input-44-846bc4624a8a>, line 36)

In [45]:
def compute_cost(Z3,Y):
    """
    计算成本
    参数：
        Z3 - 正向传播最后一个LINEAR节点的输出，维度为（6，样本数）。
        Y - 标签向量的placeholder，和Z3的维度相同
    
    返回：
        cost - 计算后的成本
    
    """
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3,labels=Y))
    
    return cost

In [48]:
def model(X_train, Y_train, X_test, Y_test, learning_rate=0.009, 
         num_epochs=100,minibatch_size=64,print_cost=True,isPlot=True):
    """
    使用TensorFlow实现三层的卷积神经网络
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    参数：
        X_train - 训练数据，维度为(None, 64, 64, 3)
        Y_train - 训练数据对应的标签，维度为(None, n_y = 6)
        X_test - 测试数据，维度为(None, 64, 64, 3)
        Y_test - 训练数据对应的标签，维度为(None, n_y = 6)
        learning_rate - 学习率
        num_epochs - 遍历整个数据集的次数
        minibatch_size - 每个小批量数据块的大小
        print_cost - 是否打印成本值，每遍历100次整个数据集打印一次
        isPlot - 是否绘制图谱
        
    返回：
        train_accuracy - 实数，训练集的准确度
        test_accuracy - 实数，测试集的准确度
        parameters - 学习后的参数
    """
    ops.reset_default_graph()
    tf.set_random_seed(1)
    seed = 3
    (m, n_H0, n_W0, n_C0) = X_train.shape
    n_y = Y_train.shape[1]
    costs = []
    
    #为当前维度创建占位符
    X, Y = create_placeholders(n_H0,b_W0,n_C0,n_y)
    
    #初始化参数
    parameters = initialize_parameters()
    
    #前向传播
    Z3 = forward_propagation(X,parameters)
    
    #计算成本
    cost = compute_cost(Z3, Y)
    
    #反向传播
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    #初始化tf变量
    init = tf.global_variables_initializer()
    
    #run
    with tf.Session() as sess:
        #初始化参数
        sess.run(init)
        #遍历数据集
        for epoch in range(num_epochs):
            minibatch_cost = 0
            num_minibatches = int(m/minibatch_size)
            seed = seed + 1
            minibatches = cnn_utils.random_mini_batches(X_train,Y_train,minibatch_size,seed)
            
            #对每个数据块进行处理
            (minibatch_X,minibatch_Y) = minibatch
            #最小化这个数据块的成本
            _, temp_cost = sess.run([optimizer,cost],feed_dict={X:minibatch_X,Y:minibatch_Y})
            
            #累加数据块的成本值
            minibatch_cost += temp_cost / num_minibatches
            
            #是否打印成本
            if print_cost:
                if epoch % 5 == 0:
                    print("当前是第 " + str(epoch) + "代，成本值为:" + str(minibatch_cost))
                    
            #记录成本
            if epoch % 1 == 0:
                costs.append(minibatch_cost)
            
            #画图
            if isPlot:
                plt.plot(np.squeeze(costs))
                plt.ylabel('cost')
                plt.xlabel('iterations (per tens)')
                plt.title("learning rate=" + str(learning_rate))
                plt.show()
                
            
            #开始预测
            ##计算当前预测情况
            predict_op = tf.arg_max(Z3,1)
            corrent_prediction = tf.equal(predict_op, tf.arg_max(Y,1))
            
            ##计算精准度
            accuracy = tf.reduce_mean(tf.cast(corrent_prediction,"float"))
            print("corrent prediction accuracy= " + str(accuracy))

            train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
            test_accuracy  = accuracy.eval({X: X_test,  Y: Y_test})
            
            print("训练集准确度: " + str(train_accuracy))
            print("测试集准确度: " + str(test_accuracy))
            
            return (train_accuracy,test_accuracy)